In [ ]:
import glob
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from gfpgan import GFPGANer
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, ImageSequenceClip, CompositeAudioClip
import os
import googletrans
from googletrans import Translator
import tensorflow as tf
import cv2
import numpy as np
from multiprocessing import Pool
import time

EXTERNAL = True
ANIME = False
SM_MODEL = True
FACE_ENHANCE = True
OLD_METHOD = False
HD_FROM_FRAMES = True
USE_RECONSTRUCT = True


if EXTERNAL:
    %store -r WIDTH
    %store -r HEIGHT
    %store -r TEXT
    %store -r VISUALIZE
    %store -r SAVE_FOLDER
    %store -r PROJECT_NAME
    %store -r SHOW_OUTPUT
    %store -r IMG_NUMBER
    %store -r THUMBNAIL
    %store -r SKIP_VIDEO
    %store -r MERGE_VIDEO
    %store -r TEXT_VOICE_GEN
    %store -r LANGUAGES
    %store -r GEN_VIDEO
    %store -r INSERT_BACKGROUND
    %store -r RECORD_FRAME_INTER_AFTER
else:
    WIDTH = 768
    HEIGHT = 768
    VISUALIZE = True  
    SAVE_FOLDER = "D:\\Deletar\\p_gen"
    PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
    STYLES_FOLDER = ".\\styles"
    SHOW_OUTPUT = True
    IMG_NUMBER = 2
    SKIP_VIDEO = []
    MERGE_VIDEO = False
    TEXT_VOICE_GEN = ['A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A']
    LANGUAGES = ['pt']
    GEN_VIDEO = True
    INSERT_BACKGROUND = False
    RECORD_FRAME_INTER_AFTER = 0
    RECORD_FRAME_LIP = -1


project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"


In [ ]:
netscale = 4
model_path_face = "./realesrgan/GFPGANv1.3.pth"
dni_weight = None
tile = 0
title_pad = 10
pre_pad = 0
half = None
gpu_id = None
upscale = 4
img_mode = 'RGBA'

print(f"Face Enhance = {FACE_ENHANCE}")

if ANIME:  
    if not SM_MODEL:
      print('loading anime 4x')
      model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
      model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"
      """ else:
      # R-ESRGAN
      model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
      model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth" """

      upsampler = RealESRGANer(
              scale=netscale,
              model_path=model_path_x4,
              dni_weight=dni_weight,
              model=model,
              tile=tile,
              tile_pad=title_pad,
              pre_pad=pre_pad,
              half= not half,
              gpu_id= gpu_id
      )

      face_enhancer = GFPGANer(
                  model_path=model_path_face,
                  upscale=upscale,
                  arch='clean',
                  channel_multiplier=2,
                  bg_upsampler=upsampler)
    else:
      print('loading anime 2x')
      model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=16, upscale=4, act_type='prelu')
      model_path_x2 = "./realesrgan/realesr-animevideov3.pth"
      upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path_x2,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
      )

      """ from gfpgan import GFPGANer """
      face_enhancer = GFPGANer(
                  model_path=model_path_face,
                  upscale=upscale,
                  arch='clean',
                  channel_multiplier=2,
                  bg_upsampler=upsampler)
        
else:
  if not SM_MODEL:
    print('loading real HD')
    from basicsr.archs.rrdbnet_arch import RRDBNet
    arch = 'clean'
    channel_multiplier = 2
    model_name = 'GFPGANv1.4'
    url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'
    model_path = os.path.join('./gfpgan/gfpgan/weights', model_name + '.pth')
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
    upsampler = RealESRGANer(
              scale=2,
              model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
              model=model,
              tile=400,
              tile_pad=10,
              pre_pad=0,
              half=True)  # need to set False in CPU mode

    face_enhancer = GFPGANer(
      model_path=model_path,
      upscale=upscale,
      arch=arch,
      channel_multiplier=channel_multiplier,
      bg_upsampler=upsampler)

  else: 
    print('loading real SM')
    model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
    file_url = [
            'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-wdn-x4v3.pth',
            'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth'
    ]
    model_path = os.path.join('weights', 'realesr-general-x4v3' + '.pth')
    if not os.path.isfile(model_path):
        ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
        for url in file_url:
            # model_path will be updated
            model_path = load_file_from_url(
                url=url, model_dir=os.path.join(ROOT_DIR, 'weights'), progress=True, file_name=None)
  
    wdn_model_path = model_path.replace('realesr-general-x4v3', 'realesr-general-wdn-x4v3')
    model_path = [model_path, wdn_model_path]
    dni_weight = [0.5, 1 - 0.5]
    
    upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
    )


    from gfpgan import GFPGANer
    face_enhancer = GFPGANer(
        model_path=model_path_face,
        upscale=upscale,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=upsampler)

def resize_image(data,size=[135, 240]):

  
  
  resized_img = tf.image.resize(
  images=data,
  size=size,   
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img


In [ ]:

if OLD_METHOD:
    if (not os.path.exists(f"{project_folder}//lip_hd")):
                        os.makedirs(f"{project_folder}//lip_hd")
    video = []
    cont = -1
    start_time = 0
    acc_time = 0
    for folder in os.listdir(f"{project_folder}//lip"):
            for filename in os.listdir(f"{project_folder}//lip//{folder}"):                   
                if (filename[-3:] == 'mp4'):
                    frames_hd = []
                    addr_video = f"{project_folder}//lip//{folder}//{filename}"                
                    video_lq_lip = VideoFileClip(addr_video)  
                    audio_lq_lip = video_lq_lip.audio        
                    if (not os.path.exists(f"{project_folder}//lip_hd//{folder}")):
                        os.makedirs(f"{project_folder}//lip_hd//{folder}")           
                    for i, frame in enumerate(video_lq_lip.iter_frames()):
                        if ANIME:                    
                            _, _, output = face_enhancer.enhance(frame, has_aligned=False, only_center_face=False, paste_back=True)
                        else:
                            _, _, restored_img = restorer.enhance(
                                resize_image(frame).numpy(),
                                has_aligned=False,
                                only_center_face=False,
                                paste_back=True,
                                weight=0.5
                            )      
                        frames_hd.append(output)
                    if (not os.path.exists(f"{project_folder}//lip_hd//{filename[:-4]}")):
                        os.makedirs(f"{project_folder}//lip_hd//{filename[:-4]}")
                    if os.path.exists(f"{project_folder}//lip_hd//{filename[:-4]}//{filename[:-4]}.mp4"):
                            os.remove(f"{project_folder}//lip_hd//{filename[:-4]}//{filename[:-4]}.mp4")   
                    video_hd_lip = ImageSequenceClip(frames_hd, fps=30)
                    video_hd_lip = video_hd_lip.set_audio(audio_lq_lip)                
                    video_hd_lip.write_videofile(f"{project_folder}//lip_hd//{filename[:-4]}//{filename[:-4]}.mp4")
    for folder in os.listdir(f"{project_folder}//lip_hd"):
            for filename in os.listdir(f"{project_folder}//lip_hd//{folder}"):
                if (filename[-3:] == 'mp4'):
                    temp_lip_video = VideoFileClip(f"{project_folder}//lip_hd//{folder}//{filename}")
                    video.append(temp_lip_video)
            merged_video = concatenate_videoclips(video, method='chain')
    if os.path.exists(f"{project_folder}//full//full_lip.mp4"):
        os.remove(f"{project_folder}//full//full_lip.mp4")
    merged_video.write_videofile(f"{project_folder}//full//full_lip.mp4")
else:
    if HD_FROM_FRAMES:
        print("WILL TAKE THE FRAMES AS INPUT")
        if (not os.path.exists(f"{project_folder}//lip_hd")):
            os.makedirs(f"{project_folder}//lip_hd")
        start = time.time_ns()
        for i, filename in enumerate(os.listdir(f"{project_folder}//lip")):
            img_path = f"{project_folder}//lip//{filename}"
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            # img = cv2.resize(img, (288, 168))
            img = cv2.resize(img, (480, 270))
            #img = resize_image(img).numpy()

            if USE_RECONSTRUCT:
                if i%10 == 0:
                    print(f"Frame: {i}")
                if FACE_ENHANCE:                    
                    _, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
                else:
                    output, _ = upsampler.enhance(img, outscale=upscale)
                    """ _, _, output = upsampler.enhance(
                        img,
                        has_aligned=False,
                        only_center_face=False,
                        paste_back=True,
                        weight=0.5
                    ) """
            else:
                output = cv2.addWeighted(img, 1 + 5/127, img, 0, 10 - 5)   
            output = cv2.resize(output, (1920, 1080))
            cv2.imwrite(f"{project_folder}//lip_hd//{filename}", output)
        end = time.time_ns()
        print(f"{int(end-start)/1000000000}s")
    else:
        print('WILL TAKE THE VIDEO AS INPUT')
        frames_hd = []
        addr_video = f"{project_folder}//full//full_lip_lq.mp4"                
        video_lq_lip = VideoFileClip(addr_video)  
        audio_lq_lip = video_lq_lip.audio                
        for i, frame in enumerate(video_lq_lip.iter_frames()):
            if i%10 == 0:
                print(f"Frame: {i}")
            if ANIME:     
                if FACE_ENHANCE:               
                    _, _, output = face_enhancer.enhance(resize_image(frame).numpy(), has_aligned=False, only_center_face=False, paste_back=True)
                else:
                    output, _ = upsampler.enhance(resize_image(frame).numpy(), outscale=upscale)
            else:
                _, _, output = restorer.enhance(
                    resize_image(frame).numpy(),
                    has_aligned=False,
                    only_center_face=False,
                    paste_back=True,
                    weight=0.5
                )  
            frames_hd.append(output)
        if os.path.exists(f"{project_folder}//lip_hd//full_lip.mp4"):
                os.remove(f"{project_folder}//lip_hd//full_lip.mp4")   
        video_hd_lip = ImageSequenceClip(frames_hd, fps=30)
        video_hd_lip = video_hd_lip.set_audio(audio_lq_lip)                
        video_hd_lip.write_videofile(f"{project_folder}//full//full_lip.mp4")
      